In [1]:
import torch
from torch import optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.io import read_image
# from torchsummary import summary

from tqdm.notebook import tqdm
import numpy as np
import os
import torchvision
import torchvision.models as models

import pandas as pd
import numpy as np
import os
import pickle
import cv2
from PIL import Image
# import matplotlib.pyplot as plt

In [2]:
torch.zeros(1).cuda()

tensor([0.], device='cuda:0')

 # Importing Data

In [3]:
PATH = 'data/'
TRAIN_PATH = PATH + 'train_images/'
train_data_path = PATH + 'train_cultivar_mapping.csv'
train_table = pd.read_csv(train_data_path)

train_table = train_table.dropna()
train_table['index_cultivar'] = train_table['cultivar'].astype('category')
train_table['index_cultivar'] = train_table['index_cultivar'].cat.codes

with open('mapping.pkl', 'rb') as f:
    index_to_cultivar = pickle.load(f)

tmp = {}
for cultivar,index in zip(train_table['cultivar'],train_table['index_cultivar']):
    if index in tmp.keys():
        assert tmp[index] == cultivar
    else:
        tmp[index] = cultivar
        
assert tmp == index_to_cultivar
train_table.head()

,image,cultivar,index_cultivar
0,2017-06-16__12-24-20-930.png,PI_257599,73
1,2017-06-02__16-48-57-866.png,PI_154987,29
2,2017-06-12__13-18-07-707.png,PI_92270,99
3,2017-06-22__13-18-06-841.png,PI_152651,6
4,2017-06-26__12-56-48-642.png,PI_176766,50


 # Creating Vanilla Submission file

In [4]:
import cv2
from PIL import Image
PATH = 'data/'
TEST_PATH = PATH + 'test/'

def apply_clahe(image):
    image = np.asarray(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    image = cv2.split(image)
    image = list(image)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    image[2] = clahe.apply(image[2])
    image = cv2.merge(image)
    image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)

    return Image.fromarray(image)

class SorghumDatasetTest(Dataset):
    def __init__(self, df, transform=None):
        self.labels = df.values
        self.image_path = TEST_PATH + df.values
        self.transform = transforms.Compose(
            [
             transforms.Resize(size=(512, 512)),
             transforms.ToTensor(),
             transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
            ])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image_name = self.labels[idx][0]
        image_path = self.image_path[idx][0]
        image = Image.open(image_path)
        image = apply_clahe(image)
        image = self.transform(image)

        return {'image':image, 'filename': image_name}
    

def create_submission_file(model):
    directory = 'data/test'
    loader = SorghumDatasetTest(pd.DataFrame(os.listdir(directory)))
    loader = torch.utils.data.DataLoader(loader,
                                         batch_size=16,
                                         shuffle=False,
                                         num_workers=8)
    submission = {"filename":[],
                  "cultivar":[]}
    #IMPORTANT######
    model.eval()
    ################
    for i, data in tqdm(enumerate(loader, 0)):
            # get the inputs
            image, filenames = data['image'],data['filename']
            with torch.no_grad():
                prediction_index = model(image.cuda()).argmax(1).cpu().detach()
            for i in range(len(filenames)):
                prediction = index_to_cultivar[prediction_index[i].item()]
                submission["filename"].append(filenames[i])
                submission["cultivar"].append(prediction)
    model = model.cpu()
    return submission


In [5]:
# modelname = 'efficientNetb4-FullSettings_pretrained'
# PATH = f'models/{modelname}'
# loading_model = torch.load(PATH)
# submision = create_submission_file(loading_model.cuda())
# sub = pd.DataFrame.from_dict(submision)
# print(sub)
# sub.to_csv(f'Submissions/{modelname}_vanilla.csv',index=False)

 # Ensembling

In this section we attempt to make the testing set more similar to the training set. To do this we do the following.
For a given testing image, we apply 15 sampled transformations (from the same distribution as in the training set). We then sum the resulting probability vectors from the evalution of these new transformed images. The prediction is the argmax of the sum of this vector.

In [7]:
import cv2
from PIL import Image
PATH = 'data/'
TEST_PATH = PATH + 'test/'
#random choice class from
#https://stackoverflow.com/questions/65447992/pytorch-how-to-apply-the-same-random-transformation-to-multiple-image

import random

class RandomChoice(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.t = random.choice(self.transforms)

    def __call__(self, img):
        return self.t(img)

#########################################################################################

def apply_clahe(image):
    image = np.asarray(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    image = cv2.split(image)
    image = list(image)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    image[2] = clahe.apply(image[2])
    image = cv2.merge(image)
    image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)

    return Image.fromarray(image)

class SorghumDatasetTestEnsemble(Dataset):
    def __init__(self, df, transform=None,number_of_transforms= 20):
        self.labels = df.values
        self.image_path = TEST_PATH + df.values
        trans = [
            transforms.RandomChoice([
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.RandomRotation(degrees=(0, 180)),
                transforms.RandomPerspective()
                
            ]),
            transforms.RandomChoice([
                transforms.RandomGrayscale(p=0.1),
                transforms.ColorJitter(brightness=.3, hue=.4),
                transforms.GaussianBlur(kernel_size=5),
                transforms.RandomInvert(p=0.1)
            ]),
            
            transforms.RandomChoice([
                transforms.RandomResizedCrop(size=(512,512),scale=(0.1,1.00)),
                transforms.Resize(size=(512, 512)),
            ]),
            
            
            
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
            # transforms.RandomErasing(inplace=True),
        ]
        self.transform = transforms.Compose(trans)
        self.n_trans = number_of_transforms

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image_name = self.labels[idx][0]
        image_path = self.image_path[idx][0]
        image = Image.open(image_path)
        image = apply_clahe(image)
        images = []
        for _ in range(self.n_trans): # number needs to be changed wrt how many images we can fit in the gpu
            image_transformed = self.transform(image)
            images.append(image_transformed.unsqueeze(0))

        
        return {'image':torch.cat(images,0), 'filename': image_name}
    

def create_submission_file_ensembling(model):
    OUTPUT_FILE = open('output.txt', 'w')
    directory = 'data/test'
    loader = SorghumDatasetTestEnsemble(pd.DataFrame(os.listdir(directory)))
    n_trans = loader.n_trans
    loader = torch.utils.data.DataLoader(loader,batch_size=1,shuffle=False,num_workers=8)
    submission = {"filename":[],"cultivar":[]}
    #IMPORTANT######
    model.eval()
    ################
    n = len(loader)
    BATCH_SIZE = 6
    for i, data in tqdm(enumerate(loader, 0),file=OUTPUT_FILE):
            OUTPUT_FILE.write(f"Iteration {i}/{n}\n")
            OUTPUT_FILE.flush()
            images, filenames = data['image'],data['filename']
            model = model.cuda()
            #do it 2 times
            images_squeezed = images.squeeze(0)
            prediction_vectors = []
            for bi in range(0,n_trans//BATCH_SIZE+1):
            
                images_batch = images_squeezed[bi*BATCH_SIZE:
                                               min((bi+1)*BATCH_SIZE,images_squeezed.shape[0])]

                prediction_indexes = model(images_batch.cuda())
                images_batch.cpu()
                prediction_indexes = prediction_indexes.cpu().detach()
                prediction_vectors.append(prediction_indexes)
            prediction_indexes = torch.concat(prediction_vectors)

            ensemble_pred = prediction_indexes.sum(0).argmax(0)
            prediction = index_to_cultivar[ensemble_pred.item()]
            submission["filename"].append(filenames[0])
            submission["cultivar"].append(prediction)
    OUTPUT_FILE.close()
    return submission



In [ ]:
modelname = 'efficientNetb4-FullSettings_pretrained'
PATH = f'models/{modelname}'
loading_model = torch.load(PATH)
submision = create_submission_file_ensembling(loading_model.cuda())
sub = pd.DataFrame.from_dict(submision)
print(sub)
sub.to_csv(f'Submissions/{modelname}_ensemble_averaging_prob_easiest_version.csv',index=False)

0it [00:00, ?it/s]

In [ ]:
import cv2
from PIL import Image
PATH = 'data/'
TEST_PATH = PATH + 'test/'
#random choice class from
#https://stackoverflow.com/questions/65447992/pytorch-how-to-apply-the-same-random-transformation-to-multiple-image

import random

class RandomChoice(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.t = random.choice(self.transforms)

    def __call__(self, img):
        return self.t(img)

#########################################################################################

def apply_clahe(image):
    image = np.asarray(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    image = cv2.split(image)
    image = list(image)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    image[2] = clahe.apply(image[2])
    image = cv2.merge(image)
    image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)

    return Image.fromarray(image)

class SorghumDatasetTestEnsemble(Dataset):
    def __init__(self, df, transform=None):
        self.labels = df.values
        self.image_path = TEST_PATH + df.values
        trans = [
            transforms.RandomChoice([
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.RandomRotation(degrees=(0, 180)),
                transforms.RandomPerspective()
                
            ]),
            transforms.RandomChoice([
                transforms.RandomGrayscale(p=0.1),
                transforms.ColorJitter(brightness=.1, hue=.4),
                transforms.GaussianBlur(kernel_size=5),
                transforms.RandomInvert(p=0.1)
            ]),
            
            transforms.RandomChoice([
                transforms.RandomResizedCrop(size=(512,512),scale=(0.1,1.00)),
                transforms.Resize(size=(512, 512)),
            ]),
            
            
            
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
            transforms.RandomErasing(inplace=True),
        ]
        self.transform = transforms.Compose(trans)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image_name = self.labels[idx][0]
        image_path = self.image_path[idx][0]
        image = Image.open(image_path)
        image = apply_clahe(image)
        images = []
        for _ in range(16):
            #TODO: COMPARE ADDING PROBS AND ADDING MAX
            image_transformed = self.transform(image)
            images.append(image_transformed.unsqueeze(0))

        
        return {'image':torch.cat(images,0), 'filename': image_name}
    

def create_submission_file_ensembling(model):
    OUTPUT_FILE = open('output.txt', 'w')
    directory = 'data/test'
    loader = SorghumDatasetTestEnsemble(pd.DataFrame(os.listdir(directory)))
    loader = torch.utils.data.DataLoader(loader,batch_size=1,shuffle=False,num_workers=8)
    submission = {"filename":[],"cultivar":[]}
    #IMPORTANT######
    model.eval()
    ################
    n = len(loader)
    for i, data in tqdm(enumerate(loader, 0),file=OUTPUT_FILE):
            OUTPUT_FILE.write(f"Iteration {i}/{n}\n")
            OUTPUT_FILE.flush()
            images, filenames = data['image'],data['filename']
            model = model.cuda()
            cuda_images = images.squeeze(0).cuda()
            prediction_indexes = model(cuda_images)
            prediction_indexes = prediction_indexes.cpu().detach()
            images = cuda_images.detach()
            second = model(cuda_images)
            print(prediction_indexes.shape)
            print(second.shape)
            prediction_indexes = torch.concatenate(prediction_indexes,
                                                   second.cpu().detach())
            print(prediction_indexes.shape)
            break
            
            # ensemble_pred = prediction_indexes.argmax(1)
            
            ensemble_pred = torch.mode(prediction_indexes.argmax(1),0).values
            prediction = index_to_cultivar[ensemble_pred.item()]
            submission["filename"].append(filenames[0])
            submission["cultivar"].append(prediction)
    OUTPUT_FILE.close()
    return submission




In [ ]:
modelname = 'efficientNetb0-FullDataAugmentation-512-retrainedv5_clahe'
PATH = f'models/{modelname}'
loading_model = torch.load(PATH)
submision = create_submission_file_ensembling(loading_model.cuda())
sub = pd.DataFrame.from_dict(submision)
print(sub)
sub.to_csv(f'Submissions/{modelname}_ensemble_maxes.csv',index=False)

 # Using features and classifiers

Not used. Bad results.

In [ ]:
# def create_submission_file_classifier(clf):
#     directory = 'data/test'
#     loader = SorghumDatasetTest(pd.DataFrame(os.listdir(directory)))
#     loader = torch.utils.data.DataLoader(loader,batch_size=16,shuffle=False,num_workers=2)
#     submission = {"filename":[],"cultivar":[]}
    
#     for i, data in tqdm(enumerate(loader, 0)):
#             # get the inputs
#             image, filenames = data['image'],data['filename']
            
#             output = model.features(image.cuda())
#             output = model.avgpool(output).cpu().detach().numpy()
#             prediction_index = clf.predict(output)
#             for i in range(len(filenames)):
#                 prediction = index_to_cultivar[prediction_index[i].item()]
#                 submission["filename"].append(filenames[i])
#                 submission["cultivar"].append(prediction)
#     return submission

In [ ]:
# with open('embeddings/efficientNetb0-FullDataAugmentation-512-retrainedv3_clahe_training_notrans.npy', 'rb') as f:
#     features = np.load(f,allow_pickle=True)
#     labels = np.load(f,allow_pickle=True)

In [ ]:
# from sklearn.svm import SVC
# from sklearn.decomposition import PCA
# clf = SVC(gamma='auto')
# pca = PCA(n_components=10)
# print("Transforming")
# features_red = pca.fit_transform(features)
# print("Fitting")
# clf.fit(features_red,labels)

In [ ]:
# clf.score(features,labels)

 # Trash code

In [ ]:
# class SorghumDataset(Dataset):
#     def __init__(self, df,train=True):
#         self.image_path = TRAIN_PATH + df['image'].values
#         self.labels = df["index_cultivar"].values
#         transformers = []
#         if train:
#             transformers += [
#              # transforms.Grayscale(3),
#              # transforms.RandomEqualize(p=1),
#              transforms.RandomHorizontalFlip(p=0.5),
#              transforms.RandomVerticalFlip(p=0.5)]
#         transformers += [
#              transforms.Resize(size=(512, 512)),
#              transforms.ToTensor(),
#              #mean computed from all coordinates
#              # transforms.Normalize(mean=[0.3445],
#              #                     std=[0.0381])
#             #mean obtained from the pytorch website for transfer learning
#             # transforms.Normalize(mean=[0.485, 0.456, 0.406],
#             #                      std=[0.229, 0.224, 0.225])
#             # mean computed for each coordinate
#                 # transforms.Normalize(mean=[0.35835076, 0.38230668, 0.29287688],
#                 #                  std=[1.81370149e-04, 1.77726425e-04, 8.24954614e-05])
#             ]
#         self.transform = transforms.Compose(transformers)

#     def __len__(self):
#         return len(self.labels)

#     def __getitem__(self, idx):
#         label = torch.tensor(self.labels[idx], dtype=torch.float32)
#         image_path = self.image_path[idx]
        
#         image = Image.open(image_path)
#         image = self.transform(image)
        
#         return {'image':image, 'target': label}
# idx = np.arange(len(train_table))
# np.random.shuffle(idx)
# cut = int(len(train_table)*0.8)
# idx_train = idx[0:cut]
# idx_val = idx[cut:]
# print(f"Train size {len(idx_train)}, test size {len(idx_val)} ")
# criterion = nn.CrossEntropyLoss()

# def accuracy(net, test_loader, cuda=True):
#     net.eval()
#     correct = 0
#     total = 0
#     # loss = 0
#     with torch.no_grad():
#         for data in test_loader:
#             images, labels = data['image'],data['target']
#             if cuda:
#                 images = images.type(torch.cuda.FloatTensor)
#                 labels = labels.type(torch.cuda.LongTensor)
#             outputs = net(images)
#             # loss+= criterion(outputs, labels).item()
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()
#           # if total > 100:
#             # break
#     net.train()
#     acc = correct / total
#     print(f'Accuracy of the network on the test images:{(100 *acc )}')
#     #return (100.0 * correct / total, loss/total)
#     return correct/total

# def train(net, optimizer, train_loader, test_loader, criterion,  n_epoch = 5,
#           train_acc_period = 100,
#           test_acc_period = 5,
#           cuda=True):
#     train_acc = []
#     test_acc = []
#     total = 0
#     for epoch in tqdm(range(n_epoch)):  # loop over the dataset multiple times
#         running_loss = 0.0
#         running_acc = 0.0
#         net.train()
#         for i, data in enumerate(train_loader, 0):
#             # get the inputs
#             inputs, labels = data['image'],data['target']
#             if cuda:
#                 inputs = inputs.type(torch.cuda.FloatTensor)
#                 labels = labels.type(torch.cuda.LongTensor)
#             # print(inputs.shape)
#             # zero the parameter gradients
#             optimizer.zero_grad()

#             # forward + backward + optimize
#             outputs = net(inputs)

#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
#             total += labels.size(0)
#             # print statistics
#             running_loss = 0.33*loss.item()/labels.size(0) + 0.66*running_loss
#             _, predicted = torch.max(outputs.data, 1)
#             correct = (predicted == labels).sum().item()/labels.size(0)
#             running_acc = 0.3*correct + 0.66*running_acc
#             if i % train_acc_period == train_acc_period-1:
#                 train_acc.append(running_acc)
#                 print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss))
#                 print('[%d, %5d] acc: %.3f' %(epoch + 1, i + 1, running_acc))
#                 running_loss = 0.0
#                 total = 0
#                 # break
                
            
#         cur_acc = accuracy(net, test_loader, cuda=cuda)
#         test_acc.append(cur_acc)
#         print('[%d] acc: %.3f' %(epoch + 1, cur_acc))

#     print('Finished Training')
#     return train_acc,test_acc

In [ ]:
# net_transfer = models.efficientnet_b2()
# net_transfer.classifier
# def net_classifier(num_classes):
#     classifier = nn.Sequential(
#         nn.Dropout(p=0.2),
#         nn.Linear(1792,num_classes)
#         # nn.ReLU(inplace=True),
#         # nn.Dropout(),
#         # nn.Linear(4096,2048),
#         # nn.ReLU(inplace=True),
#         # nn.Linear(2048, num_classes)
#     )
#     return classifier

# def net(num_classes, is_pretrained=False, **kwargs):
#     model = models.efficientnet_b4(pretrained=is_pretrained) # CHANGE MODEL HERE
#     classifier = net_classifier(num_classes)
#     model.classifier = classifier
#     return model
# training_data = SorghumDataset(train_table.iloc[idx_train])
# val_data = SorghumDataset(train_table.iloc[idx_val],train=False)
# trainloader = torch.utils.data.DataLoader(training_data,batch_size=32,shuffle=True,num_workers=8)
# valloader = torch.utils.data.DataLoader(val_data,batch_size=32,shuffle=True,num_workers=8)
# network = net(num_classes=100, pretrained=False)
# use_cuda = True
# if use_cuda and torch.cuda.is_available():
#     print("using cuda")
#     network = network.cuda()
# learning_rate = 1e-3

# #for pretrained
# # optimizer = torch.optim.Adam([
# #                 {'params': network.features.parameters(), 'lr': learning_rate/10},
# #                 {'params': network.classifier.parameters()}],
# #             lr=learning_rate)

# #for not pretrained
# optimizer = torch.optim.Adam(network.parameters(),lr=learning_rate)

# train_acc_netPretrained, test_acc_netPretrained = train(network, optimizer, trainloader, valloader, criterion,  n_epoch = 20,
#       train_acc_period = 10,
#       test_acc_period = 100)
# modelname = 'modelefficientnetb4_dataAug_fullsize_notnormalized_notPretrained'
# # modelname = 'modelefficientnetb4'
# PATH = f'models/{modelname}'
# torch.save(network, PATH)


# with open(f'plots/{modelname}.npy', 'wb') as f:
#     np.save(f, train_acc_netPretrained)
#     np.save(f, test_acc_netPretrained)